In [ ]:
import sys
sys.path.append("..")
import os
import torch
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np
import cv2
import matplotlib.pyplot as plt


In [ ]:
import utils.dataset as myDataset
import model.model as myModel
import utils.loss as myLoss


In [ ]:
arg_batchSize =32
arg_pretrainedModel = "../model/pretrainedModel/epo24facedet.pt"
arg_workers = 12
arg_dataset = "../data/"
arg_split = "train"
arg_outputPath = "../data/savings/output/"
arg_heatmapName = "_test.npy"

In [ ]:
dataset = myDataset.FaceDataset(datapath = arg_dataset, split = arg_split)
dataloader = torch.utils.data.DataLoader(dataset, shuffle = True, batch_size = arg_batchSize, \
                                         num_workers = arg_workers, drop_last = False)

In [ ]:
model = myModel.FaceKeypointModel()
model.load_state_dict(torch.load(arg_pretrainedModel))
print("Use model from " + arg_pretrainedModel)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model.eval()
model.to(device)

if not os.path.exists(arg_outputPath):
    os.makedirs(arg_outputPath)

heatmap_num = 0

In [ ]:
with torch.no_grad():
    for i, data in tqdm(enumerate(dataloader)):
        if arg_split == "train":
            image, anno, gtmap = data
            image, anno, gtmap = image.to(device,  dtype=torch.float), anno.to(device), gtmap.to(device, dtype=torch.float)
            image = image/255.0
        else:
            image = data
            image = image.to(device, dtype=torch.float)
            image = image/255.0
        
        heatMaps = model(image)
        # heatMaps = F.max_pool2d(heatMaps, 3, 1, 1, 1)
        heatmap_num = heatmap_num + 1
        
        break


In [ ]:
criterion = torch.nn.MSELoss()

In [ ]:
heatMap_img = heatMaps[1][0].cpu().numpy()
plt.imshow(heatMap_img)

plt.figure()
gtmap_img = gtmap[1][0].cpu().numpy()
plt.imshow(gtmap_img)

criterion(heatMaps[1][0], gtmap[1][0])


In [ ]:
# visualization
batch = 3

cv_img = image[batch][0].cpu().numpy()
cv_img = cv2.cvtColor(cv_img, cv2.COLOR_GRAY2BGR)

kpPos = kpPosList[batch]
for kp_center in kpPos:
    # print(kp_center)
    cv_img = cv2.circle(cv_img, kp_center, 2, (255, 0, 0), 2)

plt.imshow(cv_img)

In [ ]:
# generate submit csv
kpPosList = []

for i in range(heatMaps.shape[0]):
    heatMap = heatMaps[i]
    kpPos = []
    
    for kpID in range(15):
        heatMapkp = heatMap[kpID]
        indices = torch.argmax(heatMapkp, keepdim = True)
        y_pos = int(indices.item()/96)
        x_pos = indices.item()%96
        kpPos.append((x_pos, y_pos))

    kpPosList.append(kpPos)